In [1]:
# libraries for Twitter data collection
import pandas as pd
import tweepy
import time
from datetime import datetime, timedelta
import json
import os
import pickle

### 1st test


In [2]:
# Twitter API Configuration

BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN")
# API_KEY = os.getenv("TWITTER_API_KEY")
# API_SECRET = os.getenv("TWITTER_API_SECRET")
# ACCESS_TOKEN = os.getenv("TWITTER_ACCESS_TOKEN")
# ACCESS_TOKEN_SECRET = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")


print("API credentials found:", bool(BEARER_TOKEN))

API credentials found: True


In [ ]:
# Initialize Twitter API client
client = tweepy.Client(bearer_token=BEARER_TOKEN)
print("Twitter client created successfully")

In [ ]:
tweets_advanced = client.search_recent_tweets(
    query="Spotify AI DJ OR Spotify DJ OR #SpotifyDJ OR Spotify DJ feature -is:retweet",
    max_results=90,
    tweet_fields=[
        "created_at",
        "public_metrics",
        "author_id",
        "lang",  # Language detection
        "context_annotations",
        "referenced_tweets",
    ],
)

In [ ]:
len(tweets_advanced.data)

In [ ]:
for tweet in tweets_advanced.data:
    print(tweet.text)


In [ ]:
type(tweets_advanced)

In [ ]:
with open("tweets_advanced.pkl", "wb") as f:
    pickle.dump(tweets_advanced, f)

print("Saved raw tweets_advanced as pickle")


### 2nd try


In [3]:
BEARER_TOKEN2 = os.getenv("TWITTER_BEARER_TOKEN2")
print("API credentials found:", bool(BEARER_TOKEN2))

API credentials found: True


In [4]:
# Initialize Twitter API client
client2 = tweepy.Client(bearer_token=BEARER_TOKEN2)
print("Twitter client created successfully")


Twitter client created successfully


In [5]:
tweets_final = client2.search_recent_tweets(
    query='Spotify "AI DJ" -#ai -#dj -is:retweet lang:en',
    max_results=100,  # Max out limit
    tweet_fields=[
        "created_at",
        "public_metrics",
        "author_id",
        "lang",
        "context_annotations",
        "referenced_tweets",
    ],
)


In [ ]:
for tweet in tweets_final.data:
    print(tweet.text)


@g0dbr34th3d Your playlist is amazing for real.

Would recommend his playlist for reading, working and creating content.

Mine mostly slow rap from @1GunnaGunna and cool songs, rhymes from drake.

https://t.co/vntHjHCNGA

Drake slow and melodic rap.

https://t.co/eokqVPScvr

My Spotify AI DJ, https://t.co/ZZiLhdu32C
Makes sense that this is why the Spotify AI DJ is garbage. https://t.co/j2LUnUoR9X
I’ve only just discovered Spotify AI Dj..
my spotify AI DJ has been giving me bangers since 10pm last night
A hack to know your most listened to artist even before Spotify wrapped, is asking the AI Dj, “Play my number one artist.”
Looks like @ThePompi is still taking the lead, like he did in 2024 list. The consistency. 💯
tried the spotify AI DJ and every new song it shows me is ass
This my Spotify AI DJ too sabi wetin I like bro😊
I think my Spotify AI dj is learning that I'm autistic and only like listening to the songs I already have on my playlist
Anyone else find the @Spotify AI DJ to be h

In [7]:
with open("tweets_final.pkl", "wb") as f:
    pickle.dump(tweets_final, f)

print("Saved raw tweets_final as pickle")


Saved raw tweets_final as pickle
